# Music Genre Classification

### Data: 
* filename
* chroma_stft
* rmse
* spectral_centroid
* spectral_bandwidth
* rolloff
* zero_crossing_rate
* mfcc(1-20)

### Target
* label = blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock

Will be using a macro F1-score, as the target is well balanced


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.naive_bayes import MultinomialNB, GaussianNB 

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
data = pd.read_csv('../input/music-genre-classification/dataset.csv')
print(data.shape)
data.head()

In [ ]:
data.describe()

In [ ]:
data.isnull().any().any()

## EDA

In [ ]:
sns.countplot(data=data, y='label')

In [ ]:
plt.subplots(3, 2, figsize=(20,10))
for i, col in enumerate(['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']):
    plt.subplot(3, 2, i+1)
    sns.violinplot(data=data, x='label', y=col)

In [ ]:
plt.subplots(10, 2, figsize=(20,30))
for i in range(1,21):
    plt.subplot(10, 2, i)
    sns.violinplot(data=data, x='label', y=('mfcc'+str(i)))

# Baseline - NaiveBayes

In [ ]:
X = data.drop(columns=['filename', 'label'])
y = data.label

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')

nb_f1 = f1_score(y_test, y_pred, average='macro')
nb_f1

In [ ]:
gaus = GaussianNB()
gaus.fit(X_train, y_train)

y_pred = gaus.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')

gaus_f1 = f1_score(y_test, y_pred, average='macro')
gaus_f1

In [ ]:
f1_scores = pd.DataFrame(data={'model': ['MultinomialNB', 'GaussianNB'],
                               'f1_score': [nb_f1, gaus_f1]})
f1_scores

## Random Forest

In [ ]:
X = data.drop(columns=['filename', 'label'])
label_enc = LabelEncoder()
y = label_enc.fit_transform(data.label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
#np.array(y_test).value_counts()
np.bincount(y_test)

In [ ]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
rf_f1 = f1_score(y_test, y_pred, average='macro')
rf_f1

In [ ]:
grid_params = {'n_estimators': [75, 100, 150],
               'max_features': ['log2', 'sqrt'],
               'min_samples_split': [2, 3, 4],
               'min_samples_leaf': [1, 2, 3]}
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), 
                       param_grid=grid_params,
                       scoring='f1_macro',
                       cv=5, verbose=1, n_jobs = -1)
rf_grid.fit(X_train, y_train)
rf_grid.best_params_

In [ ]:
y_pred = rf_grid.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
rf_grid_f1 = f1_score(y_test, y_pred, average='macro')
rf_grid_f1

In [ ]:
rf_dicts = [{'model': 'RandomForestClassifier', 'f1_score': rf_f1},
            {'model': 'RandomForestClassifier + GridSearchCV', 'f1_score': rf_grid_f1}]
f1_scores = f1_scores.append(rf_dicts, ignore_index=True)
f1_scores

# Gradient Boosting

In [ ]:
gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(X_train, y_train)

In [ ]:
y_pred = gbc.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
gbc_f1 = f1_score(y_test, y_pred, average='macro')
gbc_f1

### This next cell with the GridSearchCV takes a while.

In [ ]:
grid_params = {'n_estimators': [100], #[50, 100, 150],
               'learning_rate': [0.1], #[0.06, 0.08, 0.1],
               'max_depth': [3], #[2, 3, 4]
              }
gbc_grid = GridSearchCV(GradientBoostingClassifier(random_state=42, n_iter_no_change=25), 
                        param_grid=grid_params,
                        scoring='f1_macro',
                        cv=5, verbose=3, n_jobs = -1)
gbc_grid.fit(X_train, y_train)
gbc_grid.best_params_

In [ ]:
y_pred = gbc_grid.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
gbc_grid_f1 = f1_score(y_test, y_pred, average='macro')
gbc_grid_f1

In [ ]:
gbc_dicts = [{'model': 'GradientBoostingClassifier', 'f1_score': gbc_f1},
             {'model': 'GradientBoostingClassifier + GridSearchCV', 'f1_score': gbc_grid_f1}]
f1_scores = f1_scores.append(gbc_dicts, ignore_index=True)
f1_scores

# XGBBoost
See https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning for details on hyperparameter tuning methods used here.

In [ ]:
X = data.drop(columns=['filename', 'label'])
label_enc = LabelEncoder()
y = label_enc.fit_transform(data.label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
xgb_model = XGBClassifier(random_state=42, eval_metric='merror', objective='multi:softmax', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
xgb_f1 = f1_score(y_test, y_pred, average='macro')
xgb_f1

### Using hyperopt
The way I ran it was by tuning one parameter at a time:
1. Set a range for the parameter using hp.quniform or hp.uniform.
2. Uncomment that entry in the objective() function.
3. Set the max_evals to an appropriate value in the fmin() function.  
   For example, I used 15 when testing the interger values, and typically to 100 or 200 for the gamma.
4. Plot the results. If it's obvious that there's a certain range that's performing better, reduce the range and rerun fmin().
   For example, for gamma, nearly every value below 0.5 had better results.
5. Take the best value and set it. 
6. Rinse and repeat for all the parameters.
  
**Note:** The current results were on the CPU. Setting the tree_method and setting the Kaggle Accelerator did speed up the process, but ultimately gave slighly worse results


In [ ]:
# https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning
space = {'max_depth': 4, #hp.quniform('max_depth', 2, 8, 1),
         'gamma': 0.108925004633226, #hp.uniform('gamma', 0.05, 0.35), #hp.uniform('gamma', 0, 1), 
         'reg_alpha' : 0, #hp.quniform('reg_alpha', 0, 5, 1), #hp.quniform('reg_alpha', 0, 10, 1),
         'reg_lambda' : 1, #hp.uniform('reg_lambda', 0, 2),
         'min_child_weight' : 1, #hp.quniform('min_child_weight', 0, 10, 1),
         'n_estimators': 200,
         #'tree_method': 'gpu_hist'
        }
space_plot_metric = 'gamma'

eval_metric = 'merror'
xgb_objective = 'multi:softmax'
num_class = len(data.label.unique())
num_rounds = 999
num_folds = 5
random_state = 42
early_stopping_rounds = 25
evaluation=[(X_train, y_train), (X_test, y_test)]

In [ ]:
def objective(space):
    print(space)
    clf = xgb.XGBClassifier(
        max_depth=int(space['max_depth']), 
        gamma=space['gamma'],
        reg_alpha=int(space['reg_alpha']), 
        reg_lambda=int(space['reg_lambda']), 
        min_child_weight=int(space['min_child_weight']),
        n_estimators=space['n_estimators'], 
        #tree_method=space['tree_method'],
        random_state=random_state,
        eval_metric=eval_metric, objective=xgb_objective, num_class=num_class, 
        use_label_encoder=False)

    clf.fit(X_train, y_train,
            eval_set=evaluation, 
            eval_metric=eval_metric,
            early_stopping_rounds=early_stopping_rounds,
            verbose=False)
    
    y_pred = clf.predict(X_test)
    xgb_obj_f1 = f1_score(y_test, y_pred, average='macro')

    x_vals.append(space[space_plot_metric])
    y_score.append(xgb_obj_f1)
    print (f'SCORE: {xgb_obj_f1}')
    return { 'loss': -xgb_obj_f1, 'status': STATUS_OK }

In [ ]:
trials = Trials()

x_vals = []
y_score = []
best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 1,
                        trials = trials)

In [ ]:
sns.scatterplot(x=x_vals, y=y_score)
best_hyperparams

In [ ]:
xgb_model_opt = xgb.XGBClassifier(**space,
                                  random_state=random_state,
                                  eval_metric=eval_metric, objective=xgb_objective, num_class=num_class,
                                  use_label_encoder=False)
xgb_model_opt.fit(X_train, y_train,
                  eval_set=evaluation, 
                  eval_metric=eval_metric,
                  early_stopping_rounds=50,
                  verbose=False)

In [ ]:
y_pred = xgb_model_opt.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
xgb_opt_f1 = f1_score(y_test, y_pred, average='macro')
xgb_opt_f1

In [ ]:
xgb_dicts = [{'model': 'XGBClassifier', 'f1_score': xgb_f1},
             {'model': 'XGBClassifier Optimized', 'f1_score': xgb_opt_f1}]
f1_scores.append(xgb_dicts, ignore_index=True)